In [1]:
import pandas as pd
import warnings
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble



C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def replace_strings(data,name):
    for i in range(len(data)):
        if(data[name].iloc[i].find(',')!=-1):
            data[name].iloc[i]=data[name].iloc[i].split(",")[-1]
    return data

In [3]:
Data=pd.read_excel("U:/JIRA/JIRAExport/train_data.xls")
Data.head()

,Issue Key,Summary,Status,Created,Component/s,Description,Resolved Date
0,NCG3D-75861,CLONE - When a pohne without any music connect...,Closed,2017-11-14 12:11:20,SW_INFOTAINMENT,h4. Test Environment :\n* HW Sample : P32R\n* ...,2018-01-24 10:25:31
1,NCG3D-75863,Reset when system is kept idle,Closed,2017-11-14 12:38:12,SW_MANAPPS,h4. Test Environment :\n* HW Sample : \n* Devi...,2017-11-15 07:12:04
2,NCG3D-75864,Several resets during 35 km drive,Closed,2017-11-14 12:48:53,SW_Resets,h4. Test Environment :\n* HW Sample : *dual li...,2017-11-14 14:02:45
3,NCG3D-75865,System is sometimes very sluggish when driving,Closed,2017-11-14 13:04:58,NaN,h4. Test Environment :\n* HW Sample : * dual l...,2018-08-16 06:19:38
4,NCG3D-75867,"After reaching the destination, the navi conti...",Closed,2017-11-14 13:14:17,SW_NAVIGATION,h4. Test Environment :\n* SW: 0734\n* HW Sampl...,2017-11-22 11:12:34


In [4]:
data=Data.dropna()
data=replace_strings(data,'Component/s')
warnings.simplefilter("ignore")


C:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
data.head()

,Issue Key,Summary,Status,Created,Component/s,Description,Resolved Date
0,NCG3D-75861,CLONE - When a pohne without any music connect...,Closed,2017-11-14 12:11:20,SW_INFOTAINMENT,h4. Test Environment :\n* HW Sample : P32R\n* ...,2018-01-24 10:25:31
1,NCG3D-75863,Reset when system is kept idle,Closed,2017-11-14 12:38:12,SW_MANAPPS,h4. Test Environment :\n* HW Sample : \n* Devi...,2017-11-15 07:12:04
2,NCG3D-75864,Several resets during 35 km drive,Closed,2017-11-14 12:48:53,SW_Resets,h4. Test Environment :\n* HW Sample : *dual li...,2017-11-14 14:02:45
4,NCG3D-75867,"After reaching the destination, the navi conti...",Closed,2017-11-14 13:14:17,SW_NAVIGATION,h4. Test Environment :\n* SW: 0734\n* HW Sampl...,2017-11-22 11:12:34
5,NCG3D-75765,"Map icons (compass, zoom scale and map menu) i...",Closed,2017-11-14 06:58:13,SW_NAVIGATION_JPN,h4. Test Environment :\n* HW Sample : Nissan P...,2017-12-07 12:31:42


In [6]:
Comp=list(data['Component/s'].unique())
data['Component/s']=data['Component/s'].replace(Comp,range(len(Comp)))
data.head()

,Issue Key,Summary,Status,Created,Component/s,Description,Resolved Date
0,NCG3D-75861,CLONE - When a pohne without any music connect...,Closed,2017-11-14 12:11:20,0,h4. Test Environment :\n* HW Sample : P32R\n* ...,2018-01-24 10:25:31
1,NCG3D-75863,Reset when system is kept idle,Closed,2017-11-14 12:38:12,1,h4. Test Environment :\n* HW Sample : \n* Devi...,2017-11-15 07:12:04
2,NCG3D-75864,Several resets during 35 km drive,Closed,2017-11-14 12:48:53,2,h4. Test Environment :\n* HW Sample : *dual li...,2017-11-14 14:02:45
4,NCG3D-75867,"After reaching the destination, the navi conti...",Closed,2017-11-14 13:14:17,3,h4. Test Environment :\n* SW: 0734\n* HW Sampl...,2017-11-22 11:12:34
5,NCG3D-75765,"Map icons (compass, zoom scale and map menu) i...",Closed,2017-11-14 06:58:13,4,h4. Test Environment :\n* HW Sample : Nissan P...,2017-12-07 12:31:42


In [7]:
trainDF = pd.DataFrame()
trainDF['text'] = data['Summary']
trainDF['label'] = data['Component/s']
trainDF.head()

,text,label
0,CLONE - When a pohne without any music connect...,0
1,Reset when system is kept idle,1
2,Several resets during 35 km drive,2
4,"After reaching the destination, the navi conti...",3
5,"Map icons (compass, zoom scale and map menu) i...",4


In [8]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [9]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [10]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 